In [1]:
# cd 先进入当前路径
# tensorboard --logdir logs

In [2]:
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras

# from IPython.core.interactiveshell import InteractiveShell
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
# 配置项
# # 这个要放到设置中文之前否则还是小方框
# plt.style.use("seaborn")

# # 指定默认字体 用来正常显示中文标签
# plt.rcParams['font.sans-serif'] = ['SimHei']
# # 解决保存图像是负号'-'显示为方块的问题
# plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
# InteractiveShell.ast_node_interactivity = "all"

In [4]:
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [6]:
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(128).repeat(10)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).shuffle(10000).batch(128)

In [7]:
model = keras.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10)
])

model.build(input_shape=[None, 28 * 28])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  401920    
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
dense_2 (Dense)              multiple                  32896     
_________________________________________________________________
dense_3 (Dense)              multiple                  8256      
_________________________________________________________________
dense_4 (Dense)              multiple                  2080      
_________________________________________________________________
dense_5 (Dense)              multiple                  330       
Total params: 576,810
Trainable params: 576,810
Non-trainable params: 0
__________________________________________________

In [8]:
# 创建输出日志 用于可视化
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(logdir=log_dir)

### 第一种 手动计算l1范数

In [10]:
# 优化器
optimizer = keras.optimizers.Adam(0.01)

# 统计准确率
accuracy_metrics = keras.metrics.Accuracy()

# 开始训练
for step, (x, y) in enumerate(db_train):
    x = tf.reshape(x, [-1, 28 * 28])
    with tf.GradientTape() as tap:
        y = tf.cast(y, dtype=tf.int32)
        y = tf.one_hot(y, depth=10)
        y_pred = model(x)
        loss = keras.losses.categorical_crossentropy(y, y_pred, from_logits=True)
        loss = tf.reduce_mean(loss)
        
        # 正则化
        loss_regularization = []
        for p in model.trainable_variables:
            loss_regularization.append(tf.nn.l2_loss(p))
         
        # l2正则化 左右参数相加求和
        loss_regularization = tf.reduce_sum(tf.stack(loss_regularization))
        
        loss = loss + 0.001 * loss_regularization
    
    # 对参数球梯度 更新参数
    grads = tap.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    if step % 100 == 0:
        print(step, 'loss:', float(loss), 'loss_regularization:', float(loss_regularization)) 
    
    if step % 500 == 0:
        accuracy_metrics.reset_states()
        
        for (x_, y_) in db_test:
            x_ = tf.reshape(x_, [-1, 28*28])
            y_pred = model(x_)
            y_pred = tf.argmax(y_pred, axis=1, output_type=tf.int32)
            
            accuracy_metrics.update_state(y_, y_pred)
            
        print('【正确率 accuracy_metrics = %f】' % (accuracy_metrics.result().numpy()))    
 
        with summary_writer.as_default():
            tf.summary.scalar('test_accuracy', accuracy_metrics.result().numpy(), step=step)

            
            
        
        

0 loss: 1.2199015617370605 loss_regularization: 405.07244873046875
【正确率 accuracy_metrics = 0.487200】
100 loss: 0.593031108379364 loss_regularization: 253.77618408203125
200 loss: 0.577171266078949 loss_regularization: 262.68182373046875
300 loss: 0.4717406928539276 loss_regularization: 258.01959228515625
400 loss: 0.6325197815895081 loss_regularization: 247.8853302001953
500 loss: 0.4656405746936798 loss_regularization: 270.90911865234375
【正确率 accuracy_metrics = 0.932900】
600 loss: 0.3572179675102234 loss_regularization: 261.4109802246094
700 loss: 0.5480451583862305 loss_regularization: 256.2273864746094
800 loss: 0.3889096975326538 loss_regularization: 229.26123046875
900 loss: 0.475649356842041 loss_regularization: 250.99388122558594
1000 loss: 0.3884015679359436 loss_regularization: 260.1014099121094
【正确率 accuracy_metrics = 0.948100】
1100 loss: 0.5256780385971069 loss_regularization: 273.0445556640625
1200 loss: 0.3482818901538849 loss_regularization: 249.66993713378906
1300 loss: 

### 第二种直接设置进去



In [25]:
# # 数据预处理
# def preprocess_data2(x, y):
#     x = tf.cast(x, dtype=tf.float32) / 255
#     x = tf.reshape(x, [28 * 28])
#     y = tf.cast(y, dtype=tf.int32)
#     y = tf.one_hot(y, depth=10)
#     return x, y


# # 划分训练接和测试集
batch_size = 128
db_train_ = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train_ = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)

db_test_ = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test_ = db_test.map(preprocess_data).batch(batch_size)
    

In [35]:
model2 = keras.Sequential([
    keras.layers.Dense(512, kernel_regularizer=keras.regularizers.l2(0.001), activation=tf.nn.relu, input_shape=(28*28, )),
    keras.layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'),
    keras.layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid') 
])

# model2.compile(optimizer='adam', 
#                loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
#                metrics=['accuracy'])

model2.build(input_shape=[None, 28 * 28])

model2.compile(optimizer=keras.optimizers.Adam(1e-3), 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [37]:
# model2.fit(db_train, epochs=10, validation_data=db_test, validation_freq=2)

# model2.fit(db_train, 
#           epochs=10,  # 训练次数
#           validation_freq=1, # 验证集调用的评率
#           validation_data=db_test
#           )

model2.fit(x_train, y=y_train, epochs=10)

ValueError: Error when checking input: expected dense_38_input to have 2 dimensions, but got array with shape (60000, 28, 28)

In [1]:
db_train

NameError: name 'db_train' is not defined